In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('/content/anime.csv')

print("Columns in the dataset:", df.columns)

if 'genres' in df.columns:
    df['genres'] = df['genres'].fillna('Unknown')
else:

    print("The column 'genres' does not exist. Please check the column name and update it accordingly.")
    print("Available columns:", df.columns)
    if 'genre' in df.columns:
        df['genre'] = df['genre'].fillna('Unknown')
        df['genres'] = df['genre']
    else:
        raise KeyError("No column found for 'genres' or 'genre'. Please verify the dataset.")

df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['rating'] = df['rating'].fillna(df['rating'].mean())


df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')
df['episodes'] = df['episodes'].fillna(0)

scaler = MinMaxScaler()
df[['rating', 'episodes']] = scaler.fit_transform(df[['rating', 'episodes']])

df_genres = df['genres'].str.get_dummies(sep=',')
df = pd.concat([df, df_genres], axis=1)
anime_features = df[['rating', 'episodes'] + list(df_genres.columns)]

similarity_matrix = cosine_similarity(anime_features)

def recommend_anime(target_index, similarity_matrix, top_n=5):
    """
    Recommend similar anime based on cosine similarity scores.
    :param target_index: Index of the target anime
    :param similarity_matrix: Precomputed cosine similarity matrix
    :param top_n: Number of similar anime to recommend
    :return: DataFrame of recommended anime
    """
    similar_indices = similarity_matrix[target_index].argsort()[-top_n-1:-1][::-1]
    similar_anime = df.iloc[similar_indices]
    return similar_anime[['name', 'rating', 'episodes', 'genres']]

target_index = 0
recommended_anime = recommend_anime(target_index, similarity_matrix, top_n=5)
print("\nRecommended Anime for", df.iloc[target_index]['name'])
print(recommended_anime)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Columns in the dataset: Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')
The column 'genres' does not exist. Please check the column name and update it accordingly.
Available columns: Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

Recommended Anime for Kimi no Na wa.
                                                   name    rating  episodes  \
5805                        Wind: A Breath of Heart OVA  0.561825  0.001650   
6394                       Wind: A Breath of Heart (TV)  0.536615  0.007151   
208                       Kokoro ga Sakebitagatterunda.  0.798319  0.000550   
504   Clannad: After Story - Mou Hitotsu no Sekai, K...  0.762305  0.000550   
1201                     Angel Beats!: Another Epilogue  0.715486  0.000550   

                                    genres  
5805  Drama, Romance, School, Supernatural  
6394  Drama, Romance, School, Supernatural  
208                 Drama, Ro